# 使用 Mistral 模型进行开发

## 简介

本课将涵盖：
- 探索不同的 Mistral 模型
- 了解每种模型的使用场景和适用情境
- 代码示例展示每个模型的独特功能


## Mistral 模型

在本课中，我们将探索三种不同的 Mistral 模型：**Mistral Large**、**Mistral Small** 和 **Mistral Nemo**。

这些模型都可以在 Github Model 市场免费获取。本笔记本中的代码将使用这些模型来运行代码。关于如何使用 Github Models 进行 [AI 模型原型开发](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)，可以参考这里的详细说明。


## Mistral Large 2 (2407)
Mistral Large 2 目前是 Mistral 的旗舰模型，专为企业级应用设计。

该模型是在原有 Mistral Large 的基础上升级而来，提供了
- 更大的上下文窗口——128k 对比 32k
- 更强的数学和编程任务表现——平均准确率 76.9%，而原来为 60.4%
- 多语言能力提升——支持的语言包括：英语、法语、德语、西班牙语、意大利语、葡萄牙语、荷兰语、俄语、中文、日语、韩语、阿拉伯语和印地语。

凭借这些特性，Mistral Large 在以下方面表现突出：
- *检索增强生成（RAG）*——得益于更大的上下文窗口
- *函数调用*——该模型原生支持函数调用，可以与外部工具和 API 集成。这些调用既可以并行执行，也可以按顺序依次执行。
- *代码生成*——该模型在 Python、Java、TypeScript 和 C++ 代码生成方面表现优异。


### 使用 Mistral Large 2 的 RAG 示例


在这个例子中，我们使用 Mistral Large 2 对一个文本文档运行 RAG 模式。问题是用韩语写的，询问作者在上大学之前的活动。

它使用 Cohere Embeddings Model 为文本文档和问题生成嵌入。在这个示例中，使用 faiss Python 包作为向量存储。

发送给 Mistral 模型的提示包括问题和与问题相似的检索片段。模型随后会给出自然语言的回答。


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small
Mistral Small 是 Mistral 系列中属于高端/企业级别的另一款模型。顾名思义，这是一款小型语言模型（SLM）。使用 Mistral Small 的优势包括：
- 相比 Mistral Large 和 NeMo 等 Mistral 大型语言模型，成本更低——价格下降了 80%
- 延迟低——响应速度比 Mistral 的大型语言模型更快
- 灵活性高——可以在不同环境中部署，对所需资源的限制更少

Mistral Small 非常适合以下场景：
- 基于文本的任务，比如摘要、情感分析和翻译
- 由于成本效益高，适用于需要频繁请求的应用
- 低延迟的代码相关任务，如代码审查和代码建议


## 对比 Mistral Small 和 Mistral Large

为了展示 Mistral Small 和 Large 在延迟上的差异，请运行下面的代码单元。

你应该能看到两者的响应时间相差大约 3-5 秒。同时也可以注意在相同提示下，回复的长度和风格也会有所不同。


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

与本课讨论的另外两个模型相比，Mistral NeMo 是唯一一个拥有 Apache2 许可证的免费模型。

它被认为是 Mistral 早期开源大语言模型 Mistral 7B 的升级版。

NeMo 模型还有一些其他特点：

- *更高效的分词方式：* 该模型采用 Tekken 分词器，而不是更常用的 tiktoken。这让它在多语言和代码处理方面表现更好。

- *可微调：* 基础模型支持微调。这为需要微调的应用场景提供了更大的灵活性。

- *原生函数调用* —— 和 Mistral Large 一样，这个模型也经过了函数调用训练。这使它成为首批支持该功能的开源模型之一。


## Mistral NeMo

与本课讨论的另外两个模型相比，Mistral NeMo 是唯一一个拥有 Apache2 许可证的免费模型。

它被认为是 Mistral 早期开源大语言模型 Mistral 7B 的升级版。

NeMo 模型还有一些其他特点：

- *更高效的分词方式：* 该模型采用了 Tekken 分词器，而不是更常用的 tiktoken。这让它在多种语言和代码上的表现更加出色。

- *可微调：* 基础模型支持微调。这为需要微调的应用场景提供了更大的灵活性。

- *原生函数调用* —— 和 Mistral Large 一样，这个模型也经过了函数调用的训练。作为首批支持该功能的开源模型之一，这让它非常独特。


### 对比分词器

在本示例中，我们将看看 Mistral NeMo 在分词处理上与 Mistral Large 有什么不同。

这两个示例使用相同的提示，但你会发现 NeMo 返回的分词数量比 Mistral Large 少。


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## 学习不止于此，继续你的旅程

完成本课后，欢迎访问我们的[生成式 AI 学习合集](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst)，持续提升你的生成式 AI 知识！



---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文件应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
